In [ ]:
import sys
PYTHON = %system which python
HOSTNAME = %system echo $HOSTNAME
print("python dir:     ",PYTHON)
print("HOSTNAME:       ",HOSTNAME)

sys.path.append('/home/leila/gpcr/ga/analysis/_PLOT')
sys.path.append('/home/leila/gpcr/ga/analysis/_corr_results')

from all_imports import *
from _Pearson_Corr import _r_, _common_diff, _group_, _Corr_Plotting, filtering_

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.set_option("display.max_rows", None, "display.max_columns", None)
%matplotlib inline

%load_ext autoreload
%autoreload 2

''' Author: Rezvan (Leila) Chitsazi '''

In [ ]:
pwd

In [ ]:
filenames_Lig1 = ['mdmb_dih.csv','resid_dis_L1_mdmb.csv','BS_mdmb_pia.csv','WF_toggle_switch_mdmb_pia.csv',
                  'resid_dis_L2_mdmb.csv','PIF_DRY_mdmb.csv','PackingClus_mdmb.csv','PIF_motif_mdmb_pia.csv',
                  'DRY_motif_mdmb_pia.csv','NPY_rmsd_DR_dis_mdmb.csv','actSwitch_mdmb.csv','TMs_filtered_mdmb.csv',
                  'ga_dis_mdmb.csv','TMs_ga_filtered_mdmb.csv']

combined = pd.concat([ pd.read_csv(f) for f in filenames_Lig1 ],axis=1)
combined_drop_duplicates = combined.loc[:,~combined.columns.duplicated()]
combined_drop_zero = combined_drop_duplicates.loc[:, (combined_drop_duplicates != 0).any(axis=0)]

combined_drop_zero.to_csv('_mdmb_.csv', index=False)
combined_drop_zero.head(5)

combined_drop_zero.to_csv('__mdmb__.csv', header=None, index=False)

In [ ]:
filenames_Lig2 = ['mmb_dih.csv','resid_dis_L1_mmb.csv','BS_mmb_pia.csv','WF_toggle_switch_mmb_pia.csv',
                  'resid_dis_L2_mmb.csv','PIF_DRY_mmb.csv','PackingClus_mmb.csv','PIF_motif_mmb_pia.csv',
                  'DRY_motif_mmb_pia.csv','NPY_rmsd_DR_dis_mmb.csv','actSwitch_mmb.csv','TMs_filtered_mmb.csv',
                  'ga_dis_mmb.csv','TMs_ga_filtered_mmb.csv']

combined = pd.concat([ pd.read_csv(f) for f in filenames_Lig2 ],axis=1)
combined_drop_duplicates = combined.loc[:,~combined.columns.duplicated()]
combined_drop_zero = combined_drop_duplicates.loc[:, (combined_drop_duplicates != 0).any(axis=0)]

combined_drop_zero.to_csv('_mmb_.csv', index=False)
combined_drop_zero.head(5)

combined_drop_zero.to_csv('__mmb__.csv', header=None, index=False)

In [ ]:
Lig_1 = 'mdmb'
Lig_2 = 'mmb'

whichone = 'only'

sample_1 = pd.read_csv('_' + Lig_1 + '_.csv')
sample_1_nrows = sample_1.shape[0]
sample_1_ncols = sample_1.shape[1]


sample_2 = pd.read_csv('_' + Lig_2 + '_.csv')
sample_2_nrows = sample_2.shape[0]
sample_2_ncols = sample_2.shape[1]

print('Number of distances (input) for ' + Lig_1 + ' is: ' + str(sample_1.shape[1]))
print('Number of distances (input) for ' + Lig_2 + ' is: ' + str(sample_2.shape[1]))

corr_threshold = 0.85

diff_residue_1 = 8
diff_residue_2 = 8

In [ ]:
_r_(Lig_1, sample_1_nrows, sample_1_ncols, corr_threshold, which=False)

In [ ]:
_r_(Lig_2, sample_2_nrows, sample_2_ncols, corr_threshold, which=False)

In [ ]:
_common_diff('mdmb', 'mmb', 'both', which='both')
_common_diff('mdmb', 'mmb', 'sample1', which='left_only')
_common_diff('mdmb', 'mmb', 'sample2', which='right_only')

_group_('both','L1', 'L2')       

In [ ]:
'''
Plot Correlation Matrix
_Corr_Plotting(Lig_1, Lig_2, corr_threshold)
'''

In [ ]:
def residue_filter(Lig_name, sample_name, sample_direction):
        
        """ filtering the correlation output based on the diff between residues
        sample_name: 'sample1', 'sample2', or 'both'
        sample_direction:'left_only','right_only',or 'both'
        
        'left'  : associated with 'sample1'
        'right' : associated with 'sample2'
        'both': associated with both
        'both': means common pairs between Lig_1 & Lig_2
        
        TM6 indices should always be in TM_index1_col1 & TM_index1_col2
        
        """
        
        _unduplicated = pd.read_csv(Lig_name + '_unduplicated.csv', delimiter=",") 
        _diff = pd.read_csv(sample_name + '_diff_common.csv', delimiter=",",index_col=0)


        _join = pd.merge(_unduplicated, _diff, on=["index_1","index_2"])
        _join

        if sample_direction=='both':
            
            Lig_name = Lig_name + '_both'
        else:
            Lig_name = Lig_name
        
        global Lig_1
        Lig_1 = Lig_name
        print('Ligand_name:',Lig_1)
        print('')
        
        _join['_merge'] = _join['_merge'].str.replace(sample_direction,Lig_name)
        

        df = _join.iloc[:]
        #print(df)
        print('the number of pairs considering only the r threshold filtering is: ', len(df))

        test_1 = df['index_1'].str[0:7]
        test_2 = df['index_2'].str[0:7]
        #--------------------------------------------------------------
        '''added July 17th, 2021'''
        test_1 = test_1.replace(to_replace='\(', value="", regex=True)
        test_2 = test_2.replace(to_replace='\(', value="", regex=True)
        #--------------------------------------------------------------
        
        test_1.lft = pd.to_numeric(test_1.str[:3])
        test_1.rgt = pd.to_numeric(test_1.str[-3:])

        test_2.lft = pd.to_numeric(test_2.str[:3])
        test_2.rgt = pd.to_numeric(test_2.str[-3:])
        
        #----------------------------------
        '''added July 17th, 2021'''
        test_1.lft = test_1.lft.apply(abs)
        test_1.rgt = test_1.rgt.apply(abs)
        
        test_2.lft = test_2.lft.apply(abs)
        test_2.rgt = test_2.rgt.apply(abs)
        #----------------------------------
        
        df['diff_residue1'] =  abs((test_1.lft) - (test_2.lft))
        df['diff_residue2'] =  abs((test_1.rgt) - (test_2.rgt))
        
        df['TM_index1_col1'] = test_1.lft
        df['TM_index2_col1'] = test_1.rgt
        df['TM_index1_col2'] = test_2.lft
        df['TM_index2_col2'] = test_2.rgt
        
        TM_list_1 = ['TM_index1_col1','TM_index1_col2']
        TM_list_2 = ['TM_index2_col1','TM_index2_col2']

        count = 0
        for i in range(len(TM_list_1)):
            conditions = [
                (df[TM_list_1[i]] == 999),
                
                (df[TM_list_1[i]] >= 253) & (df[TM_list_1[i]] <= 273),
                (df[TM_list_1[i]] >= 145) & (df[TM_list_1[i]] <= 150), 
                (df[TM_list_1[i]] >= 221) & (df[TM_list_1[i]] == 228),
                
                (df[TM_list_1[i]] >= 113) & (df[TM_list_1[i]] <= 120),
                (df[TM_list_1[i]] >= 121) & (df[TM_list_1[i]] <= 129),
                (df[TM_list_1[i]] >= 130) & (df[TM_list_1[i]] <= 144),
        
                (df[TM_list_1[i]] >= 175) & (df[TM_list_1[i]] <= 179),
                (df[TM_list_1[i]] >= 165) & (df[TM_list_1[i]] <= 174),
                (df[TM_list_1[i]] >= 151) & (df[TM_list_1[i]] <= 164),
        
                (df[TM_list_1[i]] >= 186) & (df[TM_list_1[i]] <= 195),
                (df[TM_list_1[i]] >= 196) & (df[TM_list_1[i]] <= 202),
                (df[TM_list_1[i]] >= 203) & (df[TM_list_1[i]] <= 220),
                
                (df[TM_list_1[i]] >= 247) & (df[TM_list_1[i]] <= 253),
                (df[TM_list_1[i]] >= 241) & (df[TM_list_1[i]] <= 246),
                (df[TM_list_1[i]] >= 229) & (df[TM_list_1[i]] <= 240),
                
                (df[TM_list_1[i]] >= 272) & (df[TM_list_1[i]] <= 280),
                (df[TM_list_1[i]] >= 281) & (df[TM_list_1[i]] <= 288),
                (df[TM_list_1[i]] >= 289) & (df[TM_list_1[i]] <= 313),
        
                (df[TM_list_1[i]] >= 363) & (df[TM_list_1[i]] <= 368),
                (df[TM_list_1[i]] >= 355) & (df[TM_list_1[i]] <= 362),
                (df[TM_list_1[i]] >= 335) & (df[TM_list_1[i]] <= 354),
        
                (df[TM_list_1[i]] >= 374) & (df[TM_list_1[i]] <= 382),
                (df[TM_list_1[i]] >= 383) & (df[TM_list_1[i]] <= 390),
                (df[TM_list_1[i]] >= 391) & (df[TM_list_1[i]] <= 414),
            ]
            count = count + 1
            values = ['Lig',
                      'ECL2','ICL1','ICL2',
                      'TM1e','TM1m','TM1i',
                      'TM2e','TM2m','TM2i',
                      'TM3e','TM3m','TM3i',
                      'TM4e','TM4m','TM4i',
                      'TM5e','TM5m','TM5i',
                      'TM6e','TM6m','TM6i',
                      'TM7e','TM7m','TM7i']
            df['index1_col_'+str(count)] = np.select(conditions, values)

        
        ct = 0
        for i in range(len(TM_list_2)):
            conditions = [
                (df[TM_list_2[i]] == 999),
                
                (df[TM_list_2[i]] >= 253) & (df[TM_list_2[i]] <= 273), 
                (df[TM_list_1[i]] >= 145) & (df[TM_list_1[i]] <= 150), 
                (df[TM_list_1[i]] >= 221) & (df[TM_list_1[i]] == 228),
                
                (df[TM_list_2[i]] >= 113) & (df[TM_list_2[i]] <= 120),
                (df[TM_list_2[i]] >= 121) & (df[TM_list_2[i]] <= 129),
                (df[TM_list_2[i]] >= 130) & (df[TM_list_2[i]] <= 144),
        
                (df[TM_list_2[i]] >= 175) & (df[TM_list_2[i]] <= 179),
                (df[TM_list_2[i]] >= 165) & (df[TM_list_2[i]] <= 174),
                (df[TM_list_2[i]] >= 151) & (df[TM_list_2[i]] <= 164),
        
                (df[TM_list_2[i]] >= 186) & (df[TM_list_2[i]] <= 195),
                (df[TM_list_2[i]] >= 196) & (df[TM_list_2[i]] <= 202),
                (df[TM_list_2[i]] >= 203) & (df[TM_list_2[i]] <= 220),
                
                (df[TM_list_2[i]] >= 247) & (df[TM_list_2[i]] <= 253),
                (df[TM_list_2[i]] >= 241) & (df[TM_list_2[i]] <= 246),
                (df[TM_list_2[i]] >= 229) & (df[TM_list_2[i]] <= 240),
                
                (df[TM_list_2[i]] >= 272) & (df[TM_list_2[i]] <= 280),
                (df[TM_list_2[i]] >= 281) & (df[TM_list_2[i]] <= 288),
                (df[TM_list_2[i]] >= 289) & (df[TM_list_2[i]] <= 313),
        
                (df[TM_list_2[i]] >= 20)  & (df[TM_list_2[i]] <= 34),
                (df[TM_list_2[i]] >= 190) & (df[TM_list_2[i]] <= 195),
                (df[TM_list_2[i]] >= 216) & (df[TM_list_2[i]] <= 220),
                (df[TM_list_2[i]] >= 256) & (df[TM_list_2[i]] <= 269),
                (df[TM_list_2[i]] >= 309) & (df[TM_list_2[i]] <= 354),
        
                (df[TM_list_2[i]] >= 374) & (df[TM_list_2[i]] <= 382),
                (df[TM_list_2[i]] >= 383) & (df[TM_list_2[i]] <= 390),
                (df[TM_list_2[i]] >= 391) & (df[TM_list_2[i]] <= 414),
            ]
            ct = ct + 1
            values = ['Lig',
                      'ECL2','ICL1','ICL2',
                      'TM1e','TM1m','TM1i',
                      'TM2e','TM2m','TM2i',
                      'TM3e','TM3m','TM3i',
                      'TM4e','TM4m','TM4i',
                      'TM5e','TM5m','TM5i',
                      'ga','ga','ga','ga','ga',
                      'TM7e','TM7m','TM7i']
            df['index2_col_'+str(ct)] = np.select(conditions, values)
        
        df['TM_residue1'] = df["index1_col_1"]+ "-" + df["index2_col_1"].map(str)
        df['TM_residue2'] = df["index1_col_2"]+ "-" + df["index2_col_2"].map(str)

        print('diff_residue1: ', diff_residue_1)
        print('diff_residue2: ', diff_residue_2)
        print('')
        
        residue_filtered = df[(df['diff_residue1'] >= diff_residue_1) | (df['diff_residue2'] >= diff_residue_2) ]
        residue_filtered
        
        residue_filtered.to_csv(Lig_name + '_residue_filtered.csv', index=False)
        
        TM_indices = df.filter(['TM_index1_col1','TM_index2_col1','TM_index1_col2','TM_index2_col2'], axis = 1)
        TM_indices.to_csv(Lig_name + '_residue_TM.csv', index=False)

In [ ]:
''' 'sample1', 'left', 'whichone=only' '''

whichone='only'
residue_filter('mdmb','sample1','left')
filtering_('mdmb','only')

In [ ]:
''' 'both', 'both', 'whichone=both' '''
whichone='both'
residue_filter('mdmb','both','both')
filtering_('mdmb_both','both')